# Assistant de Recherche pour la Génération Automatique de README

## INTRODUCTION

Dans le monde du développement logiciel, la documentation est souvent reléguée au second plan, malgré son rôle central. Un README clair et complet est pourtant la première impression qu’un utilisateur ou un contributeur aura d’un projet : il facilite la prise en main, l’installation et la compréhension des fonctionnalités. Pourtant, écrire et maintenir une documentation à jour est long, fastidieux et facilement négligé, surtout dans des projets en constante évolution.

Ce projet propose une solution révolutionnaire pour tous les développeurs : un assistant automatique capable d’analyser un projet complet et de générer un README.md structuré, cohérent et fidèle à l’état actuel du code.

Les bénéfices sont immédiats et concrets :
- Gain de temps massif : fini les heures passées à rédiger manuellement un README pour chaque projet ou chaque mise à jour.
- Documentation toujours à jour : en analysant directement le code source, l’outil élimine le phénomène de "documentation drift" où les README deviennent obsolètes par rapport au code réel.
- Accessibilité et collaboration : un README clair permet à n’importe quel collaborateur ou utilisateur de comprendre et de contribuer rapidement au projet.
- Polyvalence : même les projets multi-langages ou peu commentés peuvent bénéficier d’une documentation complète et professionnelle.

En automatisant ce processus, ce projet ne se contente pas de simplifier la vie des développeurs : il change leur manière de travailler, en rendant la documentation plus rapide, fiable et accessible. 

Imaginez des milliers de projets à travers le monde, chacun avec un README précis et clair, généré automatiquement. Ce projet a le potentiel de transformer la manière dont les développeurs documentent leurs projets, tout en améliorant la collaboration et la maintenance logicielle à grande échelle.

## Code

### Paramétrage 

1. Importation des bibliothèques nécessaires :
   
On commence par importer toutes les bibliothèques utiles pour le projet.
   

In [2]:
# Importation des bibliothèques nécessaires pour le projet.

import os            # Pour les opérations système (gestion des chemins, etc.)
import re            # Pour les expressions régulières (utilisé pour le chunking)
import json          # Pour la gestion des fichiers JSON (notamment les .ipynb)
import numpy as np   # Pour les opérations numériques (essentiel pour les embeddings)
from dotenv import load_dotenv # Pour charger la clé API depuis un fichier .env
from mistralai import Mistral # SDK officiel pour interagir avec Mistral AI
import faiss         # Bibliothèque d'indexation vectorielle (recherche d'embeddings)
import time          # Pour la gestion des pauses (retry en cas d'erreur 429)
import random        # Pour la gestion des pauses aléatoires (backoff jitter)

2. Récupération de la clé API pour l'appel à Miastral AI.

Cette étape est cruciale pour sécuriser la clé API et permettre l’accès aux services Mistral AI.

In [3]:
load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)

3. Exclusion de fichiers/dossiers inutiles

Le pipeline n’analyse que le code pertinent. On exclut donc :
- Les environnements virtuels, caches, artefacts de build, fichiers binaires.
- Les fichiers temporaires et images inutiles à l’analyse.

Cela améliore les performances et réduit le nombre de chunks inutiles.

In [4]:
# Liste des dossiers à exclure (Python, Node.js, Java, etc.)
excluded_dirs = {
    # Python
    "venv", ".venv", "__pycache__", "site-packages", "env", ".env",
    # Node.js / JS
    "node_modules", "bower_components", ".npm", ".yarn",
    # Java / JVM
    "target", "build", ".gradle", ".mvn", "out",
    # C/C++ / Rust / Go / .NET
    "cmake-build-debug", "cmake-build-release", "bin", "obj", "pkg", "dist",
    "Debug", "Release",
    # Divers IDE / SCM
    ".git", ".svn", ".hg", ".idea", ".vscode"
}

# Liste des fichiers à exclure (souvent générés automatiquement ou inutiles à l'analyse)
excluded_files = {
    # SCM / VCS
    ".gitignore", ".gitattributes", ".gitmodules",
    ".hgignore", ".svnignore",

    # Config / lockfiles
    "package-lock.json", "yarn.lock", "pnpm-lock.yaml",
    "poetry.lock", "Pipfile.lock",

    # Build / cache
    "Thumbs.db", "Desktop.ini",
    ".DS_Store",  # macOS
    "npm-debug.log", "yarn-error.log",
    "Cargo.lock", "Gemfile.lock",

    # Environnements
    ".env", ".env.local", ".env.production", ".env.development",

    # Binaires / artefacts
    "*.pyc", "*.pyo", "*.pyd",
    "*.class", "*.jar", "*.war", "*.ear",
    "*.dll", "*.so", "*.dylib",
    "*.exe", "*.out", "*.o", "*.obj",
    "*.a", "*.lib",

    # Archives
    "*.zip", "*.tar", "*.gz", "*.bz2", "*.rar",

    # Divers
    "README.md", "LICENSE", "COPYING", "CHANGELOG", "TODO", "Makefile",
    
    # Photos 
    "*.png", "*.jpg", "*.jpeg", "*.gif", "*.bmp", "*.svg"
}

## Fonctions principales

1. Affichage de l’arborescence (show_tree)

Cette fonction affiche l’arborescence du projet jusqu’au second niveau seulement.

Cela évite d’afficher des milliers de fichiers pour les gros projets. Et donc de se limiter au premier et second niveaux dans l'arborescence.

In [5]:
def show_tree(root):
    """
    Affiche :
    - Tous les fichiers du premier niveau
    - Tous les fichiers du second niveau
    - Tous les sous-dossiers du second niveau indiqués avec '...'
    - Ne descend pas au troisième niveau ou plus
    """
    lines = []

    for dirpath, dirnames, filenames in os.walk(root):
        # Filtrage des dossiers exclus
        dirnames[:] = [d for d in dirnames if d not in excluded_dirs]
        filenames[:] = [f for f in filenames if f not in excluded_files]

        # Niveau dans l'arborescence
        level = dirpath.replace(root, "").count(os.sep)
        indent = " " * 4 * level

        # Nom du dossier courant
        lines.append(f"{indent}{os.path.basename(dirpath)}/")

        if level == 0:
            # Fichiers du root
            for f in filenames:
                lines.append(f"{indent}    {f}")

            # Sous-dossiers niveau 1
            for d in dirnames:
                lines.append(f"{indent}    {d}/")

        elif level == 1:
            # Fichiers du second niveau
            for f in filenames:
                lines.append(f"{indent}    {f}")

            # Sous-dossiers du second niveau → seulement "..."
            for d in dirnames:
                lines.append(f"{indent}    {d}/ ...")

        else:
            dirnames[:] = []  # empêche d'aller plus loin

        # Empêche la descente au-delà du second niveau
        if level >= 2:
            dirnames[:] = []

    return "\n".join(lines)

2. Lecture du code (read_code)

Cette fonction lit uniquement le code exploitable :
- Pour les .ipynb, seules les cellules de type "code" sont conservées.
- Pour les autres fichiers, tout le contenu est récupéré.

In [6]:
def read_code(file_path):
    """
    Lit le contenu d'un fichier de code et renvoie uniquement le code exploitable.

    - Pour les fichiers Jupyter Notebook (.ipynb) : 
      ne conserve que le contenu des cellules de type 'code', et les concatène en une seule chaîne de caractères.
    - Pour les autres fichiers de code (.py, .js, .cpp, etc.) :
      lit l'intégralité du fichier et retourne son contenu brut.

    Paramètre :
    ----------
    file_path : str
        Chemin vers le fichier à lire.

    Retour :
    -------
    str
        Contenu du code du fichier, prêt à être utilisé pour le traitement ou
        pour la génération de chunks dans un pipeline de documentation.
    """
    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".ipynb":
        with open(file_path, "r", encoding="utf-8") as f:
            nb = json.load(f)
            content = []
            for cell in nb.get("cells", []):
                if cell.get("cell_type") == "code":
                    content.append("".join(cell.get("source", [])))
            return "\n".join(content)

    else:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()

## Découpage du code en chunks + embedding

1. Découpage du code en chunks

Nous avons donc deux fonctions qui travaillent ensemble:
- split_functions : découpe le code en fonctions ou blocs logiques.
- split_long_chunk : si un chunk est trop long, on le découpe en morceaux plus petits.

Cela permet d'assurer que chaque chunk respecte la limite maximale pour l’API d’embeddings.

In [7]:
def split_functions(code):
    """Découpe le code en fonctions/chunks (première étape de chunking logique)."""
    # Regex : recherche de motifs de fonctions 'def nom(...):' et capture leur corps.
    # re.DOTALL (ou S) est crucial pour que le '.' matche les sauts de ligne à l'intérieur de la fonction.
    pattern = r"(def [\w_]+\s*\([^)]*\):(?:\n(?:\s+.+))*)" 
    chunks = re.findall(pattern, code, re.DOTALL)
    # Si aucune fonction (ex: script simple ou code dans .ipynb), le chunk est le fichier entier.
    return chunks if chunks else [code]

In [8]:
# Taille max en caractères pour un chunk envoyé aux embeddings
MAX_CHARS_PER_CHUNK = 5000

def split_long_chunk(chunk, max_chars=MAX_CHARS_PER_CHUNK):
    """
    Si un chunk est trop long, on le découpe en morceaux plus petits basés sur les lignes.
    On essaye de garder des morceaux cohérents tout en respectant la limite de taille.
    """
    if len(chunk) <= max_chars:
        return [chunk]

    lines = chunk.splitlines()
    parts = []
    current = []
    current_len = 0

    for line in lines:
        # +1 pour le saut de ligne
        extra = len(line) + 1
        # Si on dépasse la limite, on coupe ici
        if current_len + extra > max_chars and current:
            parts.append("\n".join(current))
            current = [line]
            current_len = extra
        else:
            current.append(line)
            current_len += extra

    if current:
        parts.append("\n".join(current))

    return parts

2. Création des embeddings

Chaque chunk est transformé en vecteur numérique grâce au modèle codestral-embed.

Cela permet d'avoir une bonne gestion des erreurs 429 avec backoff exponentiel.

In [9]:
def create_embeddings(chunks, batch_size=16, max_retries=5):
    """
    Génère les vecteurs d'embeddings avec gestion du Rate Limit (429) via
    Backoff exponentiel + Jitter pour la robustesse des appels API.
    """
    embeddings = []

    for i in range(0, len(chunks), batch_size):
        # Traitement par lots (batching) pour optimiser les appels API.
        batch = chunks[i:i+batch_size] 
        for attempt in range(max_retries):
            try:
                # Appel à l'API Mistral AI (modèle codestral-embed)
                resp = client.embeddings.create( 
                    model="codestral-embed",
                    inputs=batch
                )
                # Conversion en numpy array pour l'indexation FAISS
                for emb in resp.data: 
                    embeddings.append(np.array(emb.embedding, dtype=np.float32))
                break # Succès : on sort de la boucle de retry

            except Exception as e:
                # Gestion de l'erreur 429 (Rate Limit Exceeded)
                if "429" in str(e):
                    # Calcul du temps d'attente : 2^attempt + aléatoire (Jitter)
                    wait = 2 ** attempt + random.random()
                    print(f"Erreur 429… Retry dans {wait:.2f}s")
                    time.sleep(wait)
                else:
                    raise e
    return embeddings

3. Parcours du projet (itération sur fichiers)

Cette fonction récupère seulement les fichiers code utiles, en respectant les exclusions.

In [10]:
def iter_project_files(folder_path, code_ext):
    """Itère uniquement sur les fichiers code utiles, en excluant dossiers et fichiers parasites."""
    for dirpath, dirnames, filenames in os.walk(folder_path):
        # On filtre les dossiers exclus
        dirnames[:] = [d for d in dirnames if d not in excluded_dirs]

        for f in filenames:
            # Exclure les fichiers parasites
            if f in excluded_files:
                continue
            # Exclure aussi par motif (ex: *.pyc, *.class, etc.)
            for pattern in excluded_files:
                if pattern.startswith("*.") and f.endswith(pattern[1:]):
                    break
            else:
                ext = os.path.splitext(f)[1].lower()
                if ext in code_ext:
                    yield os.path.join(dirpath, f)

## Pipeline de génération du README

Nous avons décidé de suivre cette pipeline:
1. Lecture et filtrage des fichiers code.
2. Chunking et découpage.
3. Création des embeddings.
4. Indexation FAISS pour recherche par similarité.
5. Construction du prompt RAG et génération du README via LLM.
6. Sauvegarde du README dans le dossier du projet.

Le prompt RAG structure la génération pour garantir un README complet, clair et dans l’ordre :
1. Titre du projet
2. Présentation générale
3. Arborescence
4. Bibliothèques
5. Résumé fichier par fichier
6. Instructions pour lancer le projet

In [11]:
def generate_readme_RAG(folder_path, output_file):

    # ---- 3.1 Lire fichiers code ----
    code_ext = {".py", ".js", ".ts", ".cpp", ".c", ".java", ".ipynb", ".php", ".html"}
    all_files = []
    all_codes = []

    for fp in iter_project_files(folder_path, code_ext):
        code = read_code(fp)
        if code.strip():
            all_files.append(fp)
            all_codes.append(code)

    if not all_codes:
        raise ValueError("Aucun fichier code trouvé dans ce dossier !")

  
    # ---- 3.2 Chunking ----
    chunks = []
    chunk_paths = []

    for file_path, code in zip(all_files, all_codes):
        # Première étape : découper par fonctions / blocs
        func_chunks = split_functions(code)

        # Deuxième étape : si un chunk est trop long, on le re-découpe
        for fc in func_chunks:
            small_chunks = split_long_chunk(fc, max_chars=MAX_CHARS_PER_CHUNK)
            for sc in small_chunks:
                chunks.append(sc)
                chunk_paths.append(file_path)

    print(f"Total chunks : {len(chunks)}")


    # ---- 3.3 Embeddings ----
    # Vectorisation des chunks de code.
    embeddings = create_embeddings(chunks, batch_size=16)
    dim = len(embeddings[0]) # Dimension pour l'index FAISS

    # ---- 3.4 Indexation FAISS ----
    # Création d'un index vectoriel en mémoire pour la recherche rapide par similarité.
    index = faiss.IndexFlatL2(dim) # Index FlatL2: distance euclidienne non compressée.
    index.add(np.array(embeddings)) # Indexation/Stockage des vecteurs.

    # ---- 3.5 Arborescence ----
    # Génération de l'arborescence synthétique.
    tree = show_tree(folder_path)

    # ---- 3.6 Construire le prompt RAG final ----
    # Création du prompt structuré pour guider le LLM (Codestral)
    prompt = f"""
Tu es un expert en analyse de code. 
Voici une liste de chunks extraits du projet.

Ton rôle :
- Reconstituer le sens du projet
- Faire un README.md professionnel et intuitif

Tu dois au maximum respecter exactement la structure suivante,
avec ces titres dans cet ordre, même si tu n'as pas toutes les informations :
1. # Titre du projet
2. # Présentation générale et fonctionnement
3. # Arborescence du projet
4. # Bibliothèques nécessaires pour le projet
5. # Résumé fichier par fichier
6. # Comment lancer le projet

Si tu manques d'informations pour une section, écris clairement :
"Informations à compléter" plutôt que de supprimer la section.

Voici l'arborescence du dossier :
-----------------
{tree}
-----------------

Voici un échantillon de chunks utiles (non ordonnés) :
-----------------
{chunks[:200]}
-----------------


Pour 2. # Présentation générale et fonctionnement en 20-30 lignes maximum
Pour 3. # Arborescence du projet, si elle est trop longue, réduis-la en ne gardant que les dossiers et fichiers importants.
Pour 5. # Résumé fichier par fichier, classe les par catégories de languages si possible, et pour chaque fichier, indique en 2-3 lignes son rôle dans le projet.
pOUR 6. # Comment lancer le projet, indique les étapes précises pour un utilisateur lambda.
Conclus le README par les avertissements sur le projet, et les limitations connues, en 5-10 lignes maximum.

IMPORTANT : 
- Ne mets **aucun bloc de code Markdown** autour du README.
- Ne commence PAS par ```markdown, ```md ou tout autre fence.

Génère maintenant un README Markdown complet et propre, en respectant EXACTEMENT les titres et l'ordre des sections listées plus haut.

"""


    # Appel au LLM (Codestral) pour la génération finale du README
    response = client.chat.complete(
        model="codestral-2508", 
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2, # Température basse pour une réponse factuelle et stable
        max_tokens=5000 
    )

    readme_text = response.choices[0].message.content

    # ---- 3.7 Sauvegarde ----
    # Si README.md existe, on crée README2.md
    if os.path.exists(output_file):
        base = os.path.dirname(output_file)
        output_file = os.path.join(base, "README2.md")
        print("README.md existant → génération dans README2.md")

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(readme_text)

    print("\n README généré :", output_file)


## Création du README de l'utilisateur

In [12]:
# --- Déclenchement du pipeline de génération ---
# Demande à l'utilisateur de spécifier le chemin du dossier du projet.
folder = input("Dossier à analyser : ").strip()

# Définition du chemin de sortie par défaut
output = os.path.join(folder, "README.md")

# Lancement du processus RAG + génération README
generate_readme_RAG(folder, output)

Total chunks : 26
Erreur 429… Retry dans 1.08s
Erreur 429… Retry dans 2.26s

 README généré : D:\33611\Documents\MASTER\M2\PROJET_README\creating-a-README-for-a-codebase\README.md


## Guide Utilisateur et Avertissements

Cette partie sert à guider l’utilisateur qui veut générer automatiquement un README pour son projet en utilisant ce notebook.

### Guide d’Exécution

Pour faire fonctionner la génération du README, voici les étapes à suivre :

1. Préparer l’Environnement :
    - Vérifiez que votre clé API Mistral (MISTRAL_API_KEY) est correctement configurée dans le fichier .env.
    - Assurez-vous que toutes les bibliothèques nécessaires sont installées, par exemple : numpy, mistralai, faiss-cpu, etc.

2. Lancer le Notebook :
    - Exécutez toutes les cellules du notebook dans l’ordre, de la première (imports) jusqu’à la dernière cellule de génération.

3. Indiquer le Projet à Analyser :
    - Lorsque le notebook demande “Dossier à analyser”, copiez-collez le chemin complet du dossier racine de votre projet.
      - Exemple :
          - Windows : C:\Users\NomUtilisateur\MonProjet
          - Mac/Linux : ~/Documents/MonProjet

4. Suivre l’Exécution :
Si vous voyez des erreurs 429, pas de panique !
Ce sont des limites d’API (trop de requêtes trop rapides).
Le système attend quelques secondes et réessaye automatiquement grâce au mécanisme de backoff exponentiel avec Jitter.

Vous verrez alors des messages du type :

"Total chunks : 26
Erreur 429… Retry dans 1.08s
Erreur 429… Retry dans 2.26s"

Tout est normal, l’exécution continue toute seule.

5. Récupérer le Résultat :

   - Une fois terminé, un message indique : "README généré : [chemin/vers/README.md]"

  - Le fichier README sera créé dans le dossier racine du projet.
      - Si un README existait déjà, le nouveau sera nommé README2.md.

### Avertissements et Limites Connues

Quelques points importants à connaître pour utiliser ce système correctement :

1. Qualité du Code Source :
    - Plus votre code est clair et bien structuré (variables explicites, fonctions bien nommées…), plus le README généré sera précis et utile.
    - Un code confus ou mal organisé donnera un README moins pertinent.

2. Types de Fichiers :
   - Le système est optimisé pour le code source (Python, JS, C++, etc.).
   - Certains fichiers spécifiques ou binaires (images, fichiers compilés…) ne seront pas analysés correctement.

1. Performance et Scalabilité :
   - Le temps de génération dépend de la taille du projet.
   - Pour des projets très volumineux (des milliers de fichiers), l’index FAISS en mémoire peut devenir un goulot d’étranglement, et la qualité des résultats peut diminuer.
   - Dans ces cas, il peut être nécessaire de diviser le projet en sous-dossiers ou de générer le README par parties.

## Limites liées à la Parallélisation

Dans ce projet, certaines étapes comme le chunking et la génération des embeddings vectoriels pourraient bénéficier d’une parallélisation afin d’accélérer le traitement. Cependant, plusieurs contraintes limitent l’efficacité de cette approche :

- Séquentialité obligatoire pour certaines étapes :

    L’indexation FAISS et la génération finale du README reposent sur une structure séquentielle. Tenter de paralléliser ces étapes pourrait entraîner des conflits et des incohérences dans l’index ou le résultat final.

- Coût mémoire et communication :

    La parallélisation locale augmente la consommation mémoire et nécessite une communication inter-processus plus complexe, ce qui peut contrebalancer les gains de performance.

- Granularité limitée :

    Pour de petits projets ou des chunks déjà optimisés, le bénéfice réel de la parallélisation est faible. En revanche, pour des projets très volumineux, des solutions plus sophistiquées (indexation cloud, traitement hiérarchique) peuvent offrir un vrai gain de temps.

En résumé, la parallélisation peut aider, mais elle doit être appliquée avec précaution et est surtout pertinente pour des projets de grande taille ou dans des environnements distribués.

## Conclusion

Ce projet avait pour objectif de générer automatiquement un README à partir d’un projet de code source en utilisant une architecture RAG (Retrieval-Augmented Generation). L’intégration du parsing du code, des embeddings vectoriels et d’un LLM a permis de produire une documentation structurée, cohérente et compréhensible, même lorsque le code source était peu commenté.

1. Les points forts du système sont :
   - Chunking optimisé : les fonctions et blocs de code sont découpés intelligemment pour respecter les contraintes des API et faciliter l’indexation
   - Filtrage efficace des fichiers : les fichiers inutiles ou non pertinents sont exclus, ce qui améliore la qualité globale du README
   - Prompt RAG structuré : la génération finale suit un format précis, garantissant la cohérence et la lisibilité de la documentation.

2. Cependant, certaines limites persistent :
    - La scalabilité reste un défi pour les très grands projets.
    - La mémoire nécessaire pour l’index FAISS peut devenir un goulot d’étranglement.
    - La fenêtre contextuelle des modèles limite la quantité de code pouvant être analysée simultanément.
    - La parallélisation locale présente des gains limités et doit être utilisée avec prudence.

En conclusion, ce travail démontre que la RAG est une approche efficace pour automatiser la documentation logicielle, offrant un cadre solide pour générer des README lisibles et structurés, et ouvre la voie à des améliorations futures adaptées aux projets volumineux et complexes.
